In [1]:
import json
import os

import chromadb


import autogen
from autogen import AssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

In [2]:

# Accepted file formats for that can be stored in
# a vector database instance
from autogen.retrieve_utils import TEXT_FORMATS

config_list = autogen.config_list_from_json("OAI_CONFIG_LIST.json")

assert len(config_list) > 0
print("models to use: ", [config_list[i]["model"] for i in range(len(config_list))])

models to use:  ['gpt-4.1-nano']


In [3]:
print("Accepted file formats for `docs_path`:")
print(TEXT_FORMATS)

Accepted file formats for `docs_path`:
['txt', 'json', 'csv', 'tsv', 'md', 'html', 'htm', 'rtf', 'rst', 'jsonl', 'log', 'xml', 'yaml', 'yml', 'pdf', 'mdx']


In [4]:
config_list = [
    {
        # Let's choose the Meta's Llama 3.1 model (model names must match Ollama exactly)
        "model": "llama3.1:latest",
        # We specify the API Type as 'ollama' so it uses the Ollama client class
        "api_type": "ollama",
        "stream": False,
        #"client_host": "http://192.168.0.1:11434",
    }
]

In [5]:
# 1. create an AssistantAgent instance named "assistant"
assistant = AssistantAgent(
    name="assistant",
    system_message="You are a helpful assistant.",
    llm_config={
        "timeout": 600,
        "cache_seed": 42,
        "config_list": config_list,
    },
)


In [6]:
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

# Extract text from local PDF files
pdf_files = ["book.pdf"]
#docs_content = [extract_text_from_pdf(file) for file in pdf_files]

from autogen.retrieve_utils import TEXT_FORMATS,extract_text_from_pdf
docs_content= extract_text_from_pdf("book.pdf")

In [ ]:
# Define the RetrieveUserProxyAgent instance
ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3,
    retrieve_config={
        "task": "qa",
        "docs_path": ["book.pdf"],
        "docs_content": docs_content,  # Provide the extracted content directly
        "chunk_token_size": 200,
        "model": config_list[0]["model"],
        "vector_db": "chroma",
        "overwrite": True,
        "get_or_create": True,
    },
    code_execution_config=False,
)

In [14]:
# reset the assistant. Always reset the assistant before starting a new conversation.
assistant.reset()

qa_problem = "summarize the content and use python code to any calculation given in the book"
chat_result = ragproxyagent.initiate_chat(assistant, message=ragproxyagent.message_generator, problem=qa_problem)
#agentops.end_session("Success")

File autogen-docs/book.pdf does not exist. Skipping.
2025-07-23 13:40:38,997 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Found 0 chunks.


Trying to create collection.
VectorDB returns doc_ids:  [[]]
No more context, will terminate.
ragproxyagent (to assistant):

TERMINATE

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (541dfce8-0195-4f94-be10-d0f1ece31a55): Termination message condition on agent 'assistant' met
